# Trabalho 01 - Métodos Numéricos
Alunos:

          Raphael Mauricio Sanches de jesus - 124062300
          Wladimir Augusto Pereira das Neves - 124062562
          
____
Compreendendo o problema inicial...

## Revisão da Montagem de \( A \) e \( b \)

A matriz \( A \) é definida por:
$\textbf{a}_{ij} = \frac{1}{i+j}, \quad i, j = 1, 2, \ldots, n$

O vetor \( b \) é definido como:
$\textbf{b}_{i} = \sum_{j=1}^n a_{ij}, \quad i = 1, 2, \ldots, n$

No exemplo dado para \( n = 3 \):
$\textbf
A = \begin{bmatrix}
\frac{1}{1+1} & \frac{1}{1+2} & \frac{1}{1+3} \\
\frac{1}{2+1} & \frac{1}{2+2} & \frac{1}{2+3} \\
\frac{1}{3+1} & \frac{1}{3+2} & \frac{1}{3+3}
\end{bmatrix}
= \begin{bmatrix}
\frac{1}{2} & \frac{1}{3} & \frac{1}{4} \\
\frac{1}{3} & \frac{1}{4} & \frac{1}{5} \\
\frac{1}{4} & \frac{1}{5} & \frac{1}{6}
\end{bmatrix}
$

E o vetor \( b \) é a soma dos elementos de cada linha da matriz \( A \):

$\textbf
b = \left[ \sum_{j=1}^n a_{1j}, \sum_{j=1}^n a_{2j}, \sum_{j=1}^n a_{3j} \right] = \left[ \frac{13}{12}, \frac{47}{60}, \frac{37}{60} \right]
$


## Importação das Bibliotecas e Função para Construir o Sistema

Esta célula importa as bibliotecas necessárias (`numpy` e `pandas`) e define uma função para construir a matriz \( A \) e o vetor \( b \).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def construir_sistema(n: int, dtype=np.float64) -> tuple[np.ndarray, np.ndarray]:
    # Montagem da matriz A com o tipo de dado especificado
    A = np.array([[1 / (i + j) for j in range(1, n + 1)] for i in range(1, n + 1)], dtype=dtype)
    # Montagem do vetor b como a soma dos elementos de cada linha de A
    b = np.sum(A, axis=1)
    return A, b

# Teste com uma matriz pequena (n = 3) para verificar a montagem correta
A, b = construir_sistema(3)
print("Matriz A para n=3:")
print(A)
print("\nVetor b para n=3:")
print(b)

Matriz A para n=3:
[[0.5        0.33333333 0.25      ]
 [0.33333333 0.25       0.2       ]
 [0.25       0.2        0.16666667]]

Vetor b para n=3:
[1.08333333 0.78333333 0.61666667]


**Explicação:** Esta função cria a matriz \( A \) onde cada elemento é $\textbf{a}_{\textbf{ij}} = \frac{1}{i+j+1}$  e o vetor \( b \) como a soma de cada linha da matriz \( A \).

## Implementação da Decomposição LU com Pivotamento Parcial
- Para melhorar a estabilidade numérica, é essencial implementar a decomposição LU com pivotamento parcial.

In [ ]:
def decopondo_lu_pivotando(A: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    # Inicializando as matrizes L, U e P
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n, dtype=A.dtype)
    P = np.eye(n, dtype=A.dtype)
    for k in range(n-1):
        # Encontrar o índice do maior elemento em módulo na coluna k a partir da linha k
        max_index = np.argmax(np.abs(U[k:, k])) + k
        # Trocar as linhas se necessário
        if max_index != k:
            U[[k, max_index]] = U[[max_index, k]]
            P[[k, max_index]] = P[[max_index, k]]
            if k > 0:
                L[[k, max_index], :k] = L[[max_index, k], :k]
        # Eliminação
        for i in range(k+1, n):
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:] -= L[i, k] * U[k, k:]
    return P, L, U

Explicação:

Função decompor_lu_pivotamento: Implementa a decomposição LU com pivotamento parcial, garantindo maior estabilidade numérica.

Pivotamento Parcial: Troca de linhas para posicionar o maior elemento em módulo na posição de pivô, reduzindo erros numéricos.

### Sem uso de funções externas:

In [ ]:
def decopondo_lu_pivotando_manual(A: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Realiza a decomposição LU com pivotamento parcial sem usar funções externas como np.argmax.
    """
    n = len(A)
    U = A.copy()
    L = np.eye(n)
    P = np.eye(n)

    # Percorre as colunas
    for k in range(n-1):
        # Encontrar o índice do maior valor absoluto em U na coluna k a partir da linha k
        max_index = k
        max_value = abs(U[k][k])

        for i in range(k+1, n):
            if abs(U[i][k]) > max_value:
                max_value = abs(U[i][k])
                max_index = i

        # Trocar as linhas se necessário (pivotamento)
        if max_index != k:
            # Troca as linhas de U
            U[k], U[max_index] = U[max_index].copy(), U[k].copy()

            # Troca as linhas da matriz de permutação P
            P[k], P[max_index] = P[max_index].copy(), P[k].copy()

            # Troca as linhas correspondentes em L até a coluna k
            if k > 0:
                L[k, :k], L[max_index, :k] = L[max_index, :k].copy(), L[k, :k].copy()

        # Eliminação de Gauss
        for i in range(k+1, n):
            L[i][k] = U[i][k] / U[k][k]
            for j in range(k, n):
                U[i][j] -= L[i][k] * U[k][j]

    return P, L, U

## Função para Resolver o Sistema usando a Decomposição LU com Pivotamento

In [ ]:
def resolver_sistema_lu_pivotamento(P: np.ndarray, L: np.ndarray, U: np.ndarray, b: np.ndarray) -> np.ndarray:
    # Aplicar a permutação ao vetor b
    Pb = P @ b
    # Resolução de Ly = Pb (substituição forward)
    n = L.shape[0]
    y = np.zeros_like(b)
    for i in range(n):
        y[i] = Pb[i] - L[i, :i] @ y[:i]
    # Resolução de Ux = y (substituição backward)
    x = np.zeros_like(b)
    for i in reversed(range(n)):
        if U[i, i] == 0:
            raise ZeroDivisionError("Divisão por zero na substituição backward.")
        x[i] = (y[i] - U[i, i+1:] @ x[i+1:]) / U[i, i]
    return x

Explicação:

Função resolver_sistema_lu_pivotamento: Resolve o sistema linear usando as matrizes P, L e U obtidas na decomposição LU com pivotamento.

Substituição Forward e Backward: Resolve os sistemas triangulares inferiores e superiores, respectivamente.

## Resolução do Sistema Linear e Cálculo do Número de Condicionamento

Esta célula resolve o sistema linear para cada valor de \( n \) e calcula o número de condicionamento da matriz \( A \).

☠️***VERSÃO ANTIGA, NÃO USAR!*** 🛑

In [ ]:
# Lista para armazenar os resultados - VERSÃO ANTIGA, NÃO USAR!
resultados = []

# Loop para resolver o sistema linear e calcular o número de condicionamento
for n in range(2, 31):
    A, b = construir_sistema(n)
    x_exact = np.ones(n)  # Solução exata conhecida, todos elementos iguais a 1
    x_approx = np.linalg.solve(A, b)  # Solução aproximada calculada - Ainda vou mudar pelo meu próprio algoritmo.

    # Calculando o número de condicionamento da matriz A
    cond_A = np.linalg.cond(A)

    # Perturbação para estimativa de cond(A)
    perturbacao = 1e-5
    y_tilde = x_approx + perturbacao * np.random.randn(n)  # Solução perturbada

    # Recalculando b usando a solução perturbada
    b_perturbado = A @ y_tilde

    # Estimativa de cond(A) usando a definição dada
    cond_A_estimado = np.linalg.norm(b_perturbado - b) / (perturbacao * np.linalg.norm(b))

    # Calculando o erro relativo
    erro_relativo = np.linalg.norm(x_exact - x_approx) / np.linalg.norm(x_exact)

    # Adicionando os resultados à lista
    resultados.append([n, cond_A, cond_A_estimado, erro_relativo])

# Convertendo resultados para DataFrame e exibindo
df_resultados = pd.DataFrame(resultados, columns=['n', 'cond(A)', 'Estimativa cond(A)', 'Erro Relativo'])
print(df_resultados)

     n       cond(A)  Estimativa cond(A)  Erro Relativo
0    2  3.840927e+01            0.736466   0.000000e+00
1    3  1.307161e+03            0.292008   1.909282e-14
2    4  3.926016e+04            0.281528   9.427838e-13
3    5  6.889075e+04            0.294212   3.022118e-13
4    6  1.106122e+05            0.248669   1.029459e-12
5    7  1.548166e+05            0.355825   3.222725e-12
6    8  4.530362e+06            0.600686   9.430898e-11
7    9  1.082071e+05            0.221246   1.022642e-12
8   10  5.590502e+04            0.020210   1.347071e-12
9   11  5.372282e+04            0.068035   1.623336e-12
10  12  7.728194e+04            0.130955   7.854975e-13
11  13  4.461156e+04            0.065978   8.317495e-13
12  14  6.763492e+04            0.153455   6.527377e-13
13  15  7.243809e+04            0.226831   8.804752e-13
14  16  8.151876e+04            0.017423   2.535964e-12
15  17  1.275679e+05            0.061511   1.231326e-12
16  18  3.265083e+05            0.257422   2.552

### Ao construir comente ou descomente com base na função de decomposição usada.

In [ ]:
resultados = []

for n in range(2, 31):
    # Construir o sistema com precisão simples
    A_single, b_single = construir_sistema(n, dtype=np.float32)
    x_exact = np.ones(n, dtype=np.float32)

    # Decomposição LU com pivotamento
    # P_single, L_single, U_single = decopondo_lu_pivotando(A_single)
    P_single, L_single, U_single = decopondo_lu_pivotando_manual(A_single)

    # Resolver o sistema
    x_approx_single = resolver_sistema_lu_pivotamento(P_single, L_single, U_single, b_single)

    # Calcular o erro relativo inicial
    erro_relativo_single = np.linalg.norm(x_exact - x_approx_single) / np.linalg.norm(x_exact)

    # Calcular o número de condicionamento com precisão dupla
    A_double = A_single.astype(np.float64)
    cond_A = np.linalg.cond(A_double)

    # Converter as matrizes para precisão dupla
    P_double = P_single.astype(np.float64)
    L_double = L_single.astype(np.float64)
    U_double = U_single.astype(np.float64)

    # Calcular o resíduo e resolver Ay = r com precisão dupla usando suas funções
    b_double = b_single.astype(np.float64)
    x_approx = x_approx_single.astype(np.float64)
    r = b_double - A_double @ x_approx
    y_tilde = resolver_sistema_lu_pivotamento(P_double, L_double, U_double, r)

    # Estimar cond(A)
    t = 16  # Número de dígitos significativos em precisão dupla
    cond_A_estimado = (np.linalg.norm(y_tilde) / np.linalg.norm(x_approx)) * (10 ** t)

    # Armazenar os resultados
    resultados.append([n, cond_A, cond_A_estimado, erro_relativo_single])


**Explicação:** Para cada valor de \( n \), o sistema linear é resolvido, o número de condicionamento é calculado, e uma estimativa de $\textbf{cond}\textbf(A) $ é obtida usando uma solução perturbada.

Controle de Precisão:

- Solução Inicial em Precisão Simples: A solução inicial é calculada com np.float32 para simular erros de arredondamento.

- Cálculos de Resíduo e Correção em Precisão Dupla: As matrizes e vetores são convertidos para np.float64 para melhorar a precisão nos cálculos subsequentes.


Resolução de 𝐴𝑦=𝑟: A função resolver_sistema_lu_pivotamento é usada para resolver 𝐴𝑦=𝑟, conforme solicitado.

Estimativa de cond(𝐴):

Utiliza a fórmula fornecida no exercício, garantindo que os requisitos sejam atendidos.

## Exibição da Tabela de Resultados

Esta célula converte a lista de resultados em um DataFrame do Pandas e exibe a tabela.

In [ ]:
# Convertendo resultados para DataFrame e exibindo
df_resultados = pd.DataFrame(resultados, columns=['n', 'cond(A)', 'Estimativa cond(A)', 'Erro Relativo'])
print(df_resultados)

     n       cond(A)  Estimativa cond(A)  Erro Relativo
0    2  3.847403e+01        1.296619e+03   2.827296e-07
1    3  1.353295e+03        6.020041e+09   4.229410e-06
2    4  4.588514e+04        5.195270e+10   3.096104e-04
3    5  1.544216e+06        1.178355e+12   6.271981e-04
4    6  5.830156e+07        2.260288e+14   1.515769e-01
5    7  1.568711e+09        7.753508e+16   5.164349e+01
6    8  1.238385e+09        3.163553e+15   4.356218e+01
7    9  1.310834e+09        6.525276e+16   1.984677e+02
8   10  1.057041e+09        1.597123e+16   3.447036e+01
9   11  9.986444e+08        1.150795e+16   1.653326e+01
10  12  2.367805e+09        1.813299e+16   2.283072e+01
11  13  1.719803e+09        1.704914e+16   2.815996e+01
12  14  1.256875e+09        5.623286e+16   7.100710e+01
13  15  4.046226e+09        8.768432e+16   1.424644e+02
14  16  5.960935e+10        3.114343e+16   3.172075e+01
15  17  7.822756e+09        2.257319e+16   3.226176e+01
16  18  3.673570e+09        1.409934e+16   3.836

**Explicação:** A tabela exibe o valor de \( n \), o número de condicionamento da matriz, a estimativa de  $\textbf{cond}(A)$, e o erro relativo da solução.


## Aplicação do Refinamento Iterativo

Esta célula aplica o refinamento iterativo para o sistema \( n = 30 \) para verificar se a solução aproximada pode ser melhorada.

☠️***VERSÃO ANTIGA, NÃO USAR!*** 🛑

In [ ]:
# Refinamento Iterativo para n = 30
n = 30
A, b = construir_sistema(n)
x_approx = np.linalg.solve(A, b)  # Solução inicial aproximada

# Garantindo precisão dupla
A = A.astype(np.float64)
b = b.astype(np.float64)
x_approx = x_approx.astype(np.float64)

# Calculando o resíduo com precisão máxima
r = b - A @ x_approx

# Solução do sistema Ay = r
correction = np.linalg.solve(A, r)
x_refinado = x_approx + correction

# Verificando se a solução melhorou
melhora = np.linalg.norm(x_refinado - x_approx) < np.linalg.norm(r)
print(f"A solução aproximada melhorou com o refinamento iterativo? {'Sim' if melhora else 'Não'}")

# Exibindo o resíduo e a solução refinada com precisão de 8 casas decimais
print(f"Resíduo (r) com precisão de 8 casas decimais:\n{np.round(r, 8)}")
print(f"Solução refinada (x_refinado) com precisão de 8 casas decimais:\n{np.round(x_refinado, 8)}")

A solução aproximada melhorou com o refinamento iterativo? Não
Resíduo (r) com precisão de 8 casas decimais:
[-0.  0. -0.  0. -0. -0. -0. -0.  0.  0.  0. -0.  0.  0.  0. -0. -0.  0.
 -0. -0. -0. -0.  0.  0. -0. -0.  0. -0. -0.  0.]
Solução refinada (x_refinado) com precisão de 8 casas decimais:
[   0.99998306    1.00137503    0.96369887    1.45595806   -2.17123705
   14.0062786   -30.42177274   41.21416114  -12.06346709  -26.35136666
   26.59906831   -2.45689945   11.82566512   -1.31248348  -41.258776
   62.06681305  -36.34439772   -7.6486205     9.49878662   39.66957689
   21.25454952 -103.23288244   32.73196541   36.53839936    5.89422257
  -25.058008     17.31482748  -11.72803575    5.88673444    1.12588429]


In [ ]:
n = 30
# Construir o sistema
A_single, b_single = construir_sistema(n, dtype=np.float32)

# Decomposição LU com pivotamento
# P_single, L_single, U_single = decopondo_lu_pivotando(A_single)
P_single, L_single, U_single = decopondo_lu_pivotando_manual(A_single)

# Resolver o sistema
x_approx_single = resolver_sistema_lu_pivotamento(P_single, L_single, U_single, b_single)

# Converter para precisão dupla para o refinamento
A_double = A_single.astype(np.float64)
b_double = b_single.astype(np.float64)
x_approx = x_approx_single.astype(np.float64)

# Solução exata
x_exact = np.ones(n, dtype=np.float64)

'''# Aplicar o refinamento iterativo
num_iterations = 50
for iteration in range(num_iterations):
    # Calcular o resíduo
    r = b_double - A_double @ x_approx
    # Resolver Ay = r
    delta_x = np.linalg.solve(A_double, r)
    # Atualizar a solução aproximada
    x_approx += delta_x
    # Calcular o erro relativo
    erro_relativo = np.linalg.norm(x_exact - x_approx) / np.linalg.norm(x_exact)
    print(f"Iteração {iteration+1}, Erro Relativo: {erro_relativo:e}")'''

# Aplicar o refinamento iterativo com critério de convergência
num_iterations = 50
tol = 1e-10  # Tolerância para critério de convergência

for iteration in range(num_iterations):
    # Calcular o resíduo
    r = b_double - A_double @ x_approx

    # Resolver Ay = r
    delta_x = np.linalg.solve(A_double, r)

    # Atualizar a solução aproximada
    x_approx += delta_x

    # Calcular o erro relativo
    erro_relativo = np.linalg.norm(x_exact - x_approx) / np.linalg.norm(x_exact)
    print(f"Iteração {iteration+1}, Erro Relativo: {erro_relativo:e}")

    # Verificar critério de convergência
    if np.linalg.norm(delta_x) < tol:
        print(f"Convergência atingida na iteração {iteration+1}")
        break

# Exibir a solução final com precisão de 8 casas decimais
print(f"Solução final (x_approx) com precisão de 8 casas decimais:\n{np.round(x_approx, 8)}")

Iteração 1, Erro Relativo: 2.919006e+01
Iteração 2, Erro Relativo: 2.919006e+01
Iteração 3, Erro Relativo: 2.919006e+01
Iteração 4, Erro Relativo: 2.919006e+01
Iteração 5, Erro Relativo: 2.919006e+01
Iteração 6, Erro Relativo: 2.919006e+01
Iteração 7, Erro Relativo: 2.919006e+01
Iteração 8, Erro Relativo: 2.919006e+01
Iteração 9, Erro Relativo: 2.919006e+01
Iteração 10, Erro Relativo: 2.919006e+01
Iteração 11, Erro Relativo: 2.919006e+01
Iteração 12, Erro Relativo: 2.919006e+01
Iteração 13, Erro Relativo: 2.919006e+01
Iteração 14, Erro Relativo: 2.919006e+01
Iteração 15, Erro Relativo: 2.919006e+01
Iteração 16, Erro Relativo: 2.919006e+01
Iteração 17, Erro Relativo: 2.919006e+01
Iteração 18, Erro Relativo: 2.919006e+01
Iteração 19, Erro Relativo: 2.919006e+01
Iteração 20, Erro Relativo: 2.919006e+01
Iteração 21, Erro Relativo: 2.919006e+01
Iteração 22, Erro Relativo: 2.919006e+01
Iteração 23, Erro Relativo: 2.919006e+01
Iteração 24, Erro Relativo: 2.919006e+01
Iteração 25, Erro Relativ

## Testando

In [ ]:
for iteration in range(num_iterations):
    # Calcular o resíduo
    r = b_double - A_double @ x_approx

    # Resolver Ay = r
    delta_x = np.linalg.solve(A_double, r)

    # Atualizar a solução aproximada
    x_approx += delta_x

    # Calcular o erro relativo
    erro_relativo = np.linalg.norm(x_exact - x_approx) / np.linalg.norm(x_exact)
    print(f"Iteração {iteration+1}, Erro Relativo: {erro_relativo:e}")
    print(f"Resíduo na iteração {iteration+1}: {np.linalg.norm(r)}")
    print(f"Correção (delta_x) na iteração {iteration+1}: {np.linalg.norm(delta_x)}")

    # Verificar critério de convergência
    if np.linalg.norm(delta_x) < tol:
        print(f"Convergência atingida na iteração {iteration+1}")
        break


Iteração 1, Erro Relativo: 2.919006e+01
Resíduo na iteração 1: 6.135426335717442e-15
Correção (delta_x) na iteração 1: 2.7888903972961074e-06
Iteração 2, Erro Relativo: 2.919006e+01
Resíduo na iteração 2: 4.7142009730465664e-15
Correção (delta_x) na iteração 2: 4.428665834156669e-06
Iteração 3, Erro Relativo: 2.919006e+01
Resíduo na iteração 3: 4.45059594796403e-15
Correção (delta_x) na iteração 3: 3.2244381016600056e-06
Iteração 4, Erro Relativo: 2.919006e+01
Resíduo na iteração 4: 4.039747513986448e-15
Correção (delta_x) na iteração 4: 4.088527554666953e-06
Iteração 5, Erro Relativo: 2.919006e+01
Resíduo na iteração 5: 6.3311922057810455e-15
Correção (delta_x) na iteração 5: 2.947887941139864e-06
Iteração 6, Erro Relativo: 2.919006e+01
Resíduo na iteração 6: 4.558679027077474e-15
Correção (delta_x) na iteração 6: 3.8081631744272704e-06
Iteração 7, Erro Relativo: 2.919006e+01
Resíduo na iteração 7: 5.89255511483504e-15
Correção (delta_x) na iteração 7: 2.7843961115724748e-06
Iteração 

In [ ]:
n = 30
# Construir o sistema
A_single, b_single = construir_sistema(n, dtype=np.float32)

# Decomposição LU com pivotamento
P_single, L_single, U_single = decopondo_lu_pivotando(A_single)

# Resolver o sistema
x_approx_single = resolver_sistema_lu_pivotamento(P_single, L_single, U_single, b_single)

# Converter para precisão dupla para o refinamento
A_double = A_single.astype(np.float64)
b_double = b_single.astype(np.float64)
x_approx = x_approx_single.astype(np.float64)

# Solução exata
x_exact = np.linalg.solve(A_double, b_double)  # Calcule a solução exata corretamente

# Aplicar o refinamento iterativo com critério de convergência
num_iterations = 50
tol = 1e-10  # Tolerância para critério de convergência

for iteration in range(num_iterations):
    # Calcular o resíduo
    r = b_double - A_double @ x_approx

    # Resolver Ay = r
    delta_x = np.linalg.solve(A_double, r)

    # Atualizar a solução aproximada
    x_approx += delta_x

    # Calcular o erro relativo
    erro_anterior = erro_relativo if iteration > 0 else 0  # Para a primeira iteração
    erro_relativo = np.linalg.norm(x_exact - x_approx) / np.linalg.norm(x_exact)
    print(f"Iteração {iteration+1}, Erro Relativo: {erro_relativo:e}, Diferença no erro: {erro_anterior - erro_relativo:e}")
    print(f"Resíduo na iteração {iteration+1}: {np.linalg.norm(r)}")
    print(f"Correção (delta_x) na iteração {iteration+1}: {np.linalg.norm(delta_x)}")

    # Verificar critério de convergência
    if np.linalg.norm(delta_x) < tol:
        print(f"Convergência atingida na iteração {iteration+1}")
        break

# Exibir a solução final com precisão de 8 casas decimais
print(f"Solução final (x_approx) com precisão de 8 casas decimais:\n{np.round(x_approx, 8)}")

Iteração 1, Erro Relativo: 3.976511e-08, Diferença no erro: -3.976511e-08
Resíduo na iteração 1: 7.350730230132005e-07
Correção (delta_x) na iteração 1: 188.99685180619474
Iteração 2, Erro Relativo: 2.318306e-08, Diferença no erro: 1.658205e-08
Resíduo na iteração 2: 4.435337509586951e-15
Correção (delta_x) na iteração 2: 3.0653584737179916e-06
Iteração 3, Erro Relativo: 1.663668e-08, Diferença no erro: 6.546374e-09
Resíduo na iteração 3: 5.282608685317633e-15
Correção (delta_x) na iteração 3: 4.5519661688756795e-06
Iteração 4, Erro Relativo: 2.400698e-08, Diferença no erro: -7.370301e-09
Resíduo na iteração 4: 4.890025172686938e-15
Correção (delta_x) na iteração 4: 4.415340375255693e-06
Iteração 5, Erro Relativo: 6.791733e-08, Diferença no erro: -4.391034e-08
Resíduo na iteração 5: 3.643494191112748e-15
Correção (delta_x) na iteração 5: 7.314149192276978e-06
Iteração 6, Erro Relativo: 2.447377e-08, Diferença no erro: 4.344356e-08
Resíduo na iteração 6: 6.2842938400131246e-15
Correção 

In [ ]:
# Converter para precisão dupla para o refinamento
A_double = A_single.astype(np.float64)
b_double = b_single.astype(np.float64)
x_approx = x_approx_single.astype(np.float64)

# Solução exata
x_exact = np.linalg.solve(A_double, b_double)  # Calcule a solução exata corretamente

# Verificar o número de condicionamento de A
cond_A = np.linalg.cond(A_double)
print(f"Número de Condicionamento de A: {cond_A:e}")

Número de Condicionamento de A: 7.980832e+09


In [ ]:
# Aplicar o refinamento iterativo com critério de convergência
num_iterations = 50
tol = 1e-10  # Tolerância para critério de convergência
tol_diff = 1e-8  # Tolerância para a diferença no erro relativo

for iteration in range(num_iterations):
    # Calcular o resíduo
    r = b_double - A_double @ x_approx

    # Resolver Ay = r
    delta_x = np.linalg.solve(A_double, r)

    # Atualizar a solução aproximada
    x_approx += delta_x

    # Calcular o erro relativo
    erro_anterior = erro_relativo if iteration > 0 else 0  # Para a primeira iteração
    erro_relativo = np.linalg.norm(x_exact - x_approx) / np.linalg.norm(x_exact)
    print(f"Iteração {iteration+1}, Erro Relativo: {erro_relativo:e}, Diferença no erro: {erro_anterior - erro_relativo:e}")
    print(f"Resíduo na iteração {iteration+1}: {np.linalg.norm(r)}")
    print(f"Correção (delta_x) na iteração {iteration+1}: {np.linalg.norm(delta_x)}")

    # Verificar critério de convergência baseado em delta_x
    if np.linalg.norm(delta_x) < tol:
        print(f"Convergência atingida com correção pequena na iteração {iteration+1}")
        break

    # Verificar critério de parada com base na diferença no erro relativo
    if abs(erro_anterior - erro_relativo) < tol_diff:
        print(f"Convergência atingida com pequena diferença no erro relativo na iteração {iteration+1}")
        break

Iteração 1, Erro Relativo: 3.976511e-08, Diferença no erro: -3.976511e-08
Resíduo na iteração 1: 7.350730230132005e-07
Correção (delta_x) na iteração 1: 188.99685180619474
Iteração 2, Erro Relativo: 2.318306e-08, Diferença no erro: 1.658205e-08
Resíduo na iteração 2: 4.435337509586951e-15
Correção (delta_x) na iteração 2: 3.0653584737179916e-06
Iteração 3, Erro Relativo: 1.663668e-08, Diferença no erro: 6.546374e-09
Resíduo na iteração 3: 5.282608685317633e-15
Correção (delta_x) na iteração 3: 4.5519661688756795e-06
Convergência atingida com pequena diferença no erro relativo na iteração 3


## Testando a solução
Tentativa de validar a solução e comparar com metodo solve.

In [ ]:
n = 30
# Construir o sistema
A_single, b_single = construir_sistema(n, dtype=np.float32)

# Decomposição LU com pivotamento
# P_single, L_single, U_single = decopondo_lu_pivotando(A_single)
P_single, L_single, U_single = decopondo_lu_pivotando_manual(A_single)

# Resolver o sistema diretamente usando np.linalg.solve (solução aproximada inicial)
x_approx_single = resolver_sistema_lu_pivotamento(P_single, L_single, U_single, b_single)

# Converter para precisão dupla para o refinamento
A_double = A_single.astype(np.float64)
b_double = b_single.astype(np.float64)
x_approx = x_approx_single.astype(np.float64)

# Solução exata com np.linalg.solve
x_exact = np.linalg.solve(A_double, b_double)

# Aplicar o refinamento iterativo
num_iterations = 50
tol = 1e-10
tol_diff = 1e-8

for iteration in range(num_iterations):
    # Calcular o resíduo
    r = b_double - A_double @ x_approx  # Calcula quanto a solução atual está longe de resolver o sistema

    # Resolver o sistema A * delta_x = r para obter a correção
    delta_x = np.linalg.solve(A_double, r)

    # Atualizar a solução aproximada
    x_approx += delta_x

    # Calcular o erro relativo
    erro_anterior = erro_relativo if iteration > 0 else 0
    erro_relativo = np.linalg.norm(x_exact - x_approx) / np.linalg.norm(x_exact)
    print(f"Iteração {iteration+1}, Erro Relativo: {erro_relativo:e}, Diferença no erro: {erro_anterior - erro_relativo:e}")

    # Verificar critério de convergência com base no delta_x
    if np.linalg.norm(delta_x) < tol:
        print(f"Convergência atingida com correção pequena na iteração {iteration+1}")
        break

    # Verificar critério de parada com base na diferença no erro relativo
    if abs(erro_anterior - erro_relativo) < tol_diff:
        print(f"Convergência atingida com pequena diferença no erro relativo na iteração {iteration+1}")
        break

# Comparar as soluções
print(f"Solução exata:\n{x_exact}")
print(f"Solução aproximada após refinamento:\n{x_approx}")

Iteração 1, Erro Relativo: 1.631013e-08, Diferença no erro: -1.631013e-08
Iteração 2, Erro Relativo: 2.321988e-08, Diferença no erro: -6.909751e-09
Convergência atingida com pequena diferença no erro relativo na iteração 2
Solução exata:
[  0.98105916   1.33758641  -0.90709785   5.93580964  -4.42724101
  -7.68496325  40.99888201 -30.0850609  -38.40525468  44.83399798
   6.76984717   2.67211436  20.0921645  -32.64078177  16.50781678
  -4.47384065 -32.49384947  -8.76575254  13.32064113  54.42108677
  -7.77788535 -46.43562931  35.39000121  46.93668052 -51.4690238
 -18.1674198   42.22117819 -13.22060531 -39.42735835  33.96389223]
Solução aproximada após refinamento:
[  0.98105916   1.33758638  -0.90709765   5.93580919  -4.42724107
  -7.68496186  40.99888107 -30.08506199 -38.40525377  44.83399811
   6.7698474    2.67211417  20.09216387 -32.6407808   16.50781694
  -4.47384127 -32.49384945  -8.76575264  13.32064143  54.42108658
  -7.77788657 -46.43562855  35.39000258  46.93667954 -51.46902396

## Conclusão e Análise dos Resultados:

### 1. Resolva o sistema linear Ax = b considerando n = 2, 3, 4, ..., 30 e descreva o que está acontecendo.
- Construção da função `construir_sistema()` para montar a matriz \(A\) e o vetor \(b\) e a função `resolver_sistema_lu_pivotamento()` para resolver o sistema linear \(Ax = b\) usando a decomposição LU com pivotamento parcial. Implementando um loop para resolver o sistema para valores de \(n\) de 2 a 30. Isso está de acordo com o que foi requisitado na questão.
  
### 2. Utilize o numpy para calcular o número de condicionamento da matriz A para os diversos valores de n do item anterior.
- Solução utilizando `np.linalg.cond()` para calcular o número de condicionamento da matriz \(A\), armazenando esses valores em uma lista que é posteriormente convertida em um DataFrame do pandas para exibição dos resultados. Isso responde à segunda questão.

### 3. Utilize o numpy para estimar o valor de cond(A), ou seja, considerando que os cálculos para resolver o sistema linear utilizam um sistema aritmético de t dígitos significativos.
- Implementando uma estimativa de `cond(A)` baseada no erro de arredondamento utilizando a fórmula mencionada na questão:

$$
\text{cond}(A) \approx \frac{||\tilde{y}||}{||\tilde{x}||} \times 10^t
$$

A função utiliza `np.linalg.norm` para calcular as normas e, assim, estimar `cond(A)`. Isso cobre a terceira questão.


### 4. Monte uma tabela com as colunas valor de n, cond(A), estimativa de cond(A) e erro relativo da solução numérica, para n = 2, 3, 4, ..., 30.
- Usando a solução para montar um DataFrame com as colunas \(n\), \( \text{cond}(A) \), estimativa de \( \text{cond}(A) \), e o erro relativo da solução numérica. Isso corresponde ao que a quarta questão pede.

### 5. Aplique o refinamento iterativo na solução aproximada encontrada para o sistema 30 x 30. A solução aproximada melhorou?
- Na última parte, é implementado o refinamento iterativo para o sistema com \(n = 30\), atualizando a solução aproximada com base no resíduo e verificando se houve melhoria. A pergunta é respondida se a solução melhorou ou não, conforme o refinamento iterativo foi aplicado.

